# 1. 이미지를 제대로 열기 위해서는 이 함수를 적용하시오

In [6]:
import streamlit as st
import os
import json
import re
from PIL import Image, ImageDraw, ImageFont, ExifTags


In [7]:
def open_image_correct_orientation(image_path):
    # 이미지 열기
    image = Image.open(image_path)

    # EXIF 데이터에서 Orientation 태그 확인
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = image._getexif()
        if exif is not None and orientation in exif:
            if exif[orientation] == 3:
                image = image.rotate(180, expand=True)
            elif exif[orientation] == 6:
                image = image.rotate(270, expand=True)
            elif exif[orientation] == 8:
                image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # EXIF 데이터가 없거나 처리할 수 없는 경우
        pass

    return image


# 2. preprocessing 한 train_json

In [ ]:
import os
import json

# 기본 경로 설정
data_path = 'data/'
languages = {
    'Chinese': 'chinese_receipt',
    'Japanese': 'japanese_receipt',
    'Thai': 'thai_receipt',
    'Vietnamese': 'vietnamese_receipt'
}

def remove_empty_transcriptions(data):
    """
    모든 extractor에서 transcription이 빈 문자열인 항목을 제거하는 함수.
    
    Parameters:
    data (dict): train_data와 같은 이미지 데이터를 담고 있는 딕셔너리.
    """
    for extractor_key, extractor_data in data["images"].items():
        words = extractor_data.get("words", {})
        
        # 각 extractor의 words에서 최대 key 값을 찾음
        max_key = max(int(key) for key in words.keys() if key.isdigit())
        
        # 빈 문자열 transcription을 가진 키를 수집
        keys_to_remove = [
            key for key, word_data in words.items()
            if key.isdigit() and 1 <= int(key) <= max_key and word_data["transcription"] == ""
        ]
        
        # 수집된 키를 딕셔너리에서 제거
        for key in keys_to_remove:
            del words[key]

# 각 언어별로 train.json을 처리하여 현재 디렉터리에 저장
for language, folder in languages.items():
    ufo_path = os.path.join(data_path, folder, "ufo")
    train_json_path = os.path.join(ufo_path, 'train.json')
    
    # JSON 파일 로드
    with open(train_json_path, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    
    # 빈 transcription 항목 제거
    remove_empty_transcriptions(train_data)
    
    # 결과를 현재 디렉터리에 새로운 파일로 저장 (언어 이름을 파일명에 반영)
    output_path = f'preprocessing_{language}_train.json'
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)
    
    print(f"{language}의 preprocessing_{language}_train.json 파일이 현재 디렉터리에 저장되었습니다.")
